# TPC2: Repetitive Line Remover Filter

This script is a command-line utility that removes duplicate lines from a text file (or standard input). It provides three modes:

1. Default Mode: Outputs unique lines, maintaining their order.
2. `--unique` Mode: Outputs the line numbers of unique (non-duplicate) lines.
3. `--duplicate` Mode: Outputs the line numbers of duplicate lines.

Furthermore, it allows you to keep the empty lines, match against stripped text, or comment the duplicated lines.

## Code

This notebook has been designed to allow the export of the entire code with the following shell script:

```shell
jupyter nbconvert --to python --TagRemovePreprocessor.remove_input_tags="test" rl.ipynb --output rl.py
chmod +x rl.py
```

This way, the test cells are removed. Furthermore, the `README.md` can be generated by running the following shell script:

```shell
jupyter nbconvert --to markdown --execute rl.ipynb --output README.md
```

In [1]:
#!/usr/bin/env python3
import sys
import argparse

Let's start by writing the main code for processing the lines. It simply works by keeping track of the unique lines in the set. In this code we are using `yield` so that the return of the function is a stream iterator, allowing the stdin to work as expected.

In [2]:
def process_lines(lines, _, mode, use_spacing, ignores_empty, comment):
    line_set = set()

    for idx, line in enumerate(lines, start=1):
        line = line.rstrip('\n')

        is_duplicate = line in line_set
        if use_spacing:
            is_duplicate = line.strip() in line_set
        if ignores_empty and line.strip() == '':
            is_duplicate = False

        if is_duplicate:
            if mode == 'duplicate':
                yield str(idx)
            elif mode == 'comment':
                yield comment + line
        else:
            line_set.add(line)

            if mode == 'unique':
                yield str(idx)
            elif mode in ['default', 'comment']:
                yield line

In [3]:
sample = '''
Lorem ipsum odor amet, consectetuer adipiscing elit.
Accumsan at consectetur egestas scelerisque interdum lorem.
Libero class libero litora nam aliquet facilisis iaculis elementum dolor.
Malesuada laoreet tortor nam posuere eget sagittis varius. Molestie donec ad metus orci a.
Parturient class dolor molestie lacus dui ut feugiat.
Conubia curae elementum parturient faucibus accumsan.
Sed ante vestibulum platea himenaeos rhoncus sollicitudin mollis.

Lorem ipsum odor amet, consectetuer adipiscing elit.
Sed ante vestibulum platea himenaeos rhoncus sollicitudin mollis.
'''
sample_lines = sample.splitlines(keepends=True)

print('default:', list(process_lines(sample_lines, None, 'default', False, False, None))) # should output the text from lines [1,9[
print('unique:', list(process_lines(sample_lines, None, 'unique', False, False, None))) # should output [1, 2, 3, 4, 5, 6, 7, 8]
print('duplicate:', list(process_lines(sample_lines, None, 'duplicate', False, False, None))) # should output [9, 10, 11]

default: ['', 'Lorem ipsum odor amet, consectetuer adipiscing elit.', 'Accumsan at consectetur egestas scelerisque interdum lorem.', 'Libero class libero litora nam aliquet facilisis iaculis elementum dolor.', 'Malesuada laoreet tortor nam posuere eget sagittis varius. Molestie donec ad metus orci a.', 'Parturient class dolor molestie lacus dui ut feugiat.', 'Conubia curae elementum parturient faucibus accumsan.', 'Sed ante vestibulum platea himenaeos rhoncus sollicitudin mollis.']
unique: ['1', '2', '3', '4', '5', '6', '7', '8']
duplicate: ['9', '10', '11']


Next, we must deal with the cli arguments, for that we are going to use the argparse python library. By default, it uses the `sys.stdin` and `sys.stdout` as the input and output, but that can be changed by providing the filename as an argument and using the `-o` and providing the output filename. Furthermore, the mutually exclusive `--unique` and `--duplicate` will toggle the used mode.

In [30]:
def parse_args():
    parser = argparse.ArgumentParser(
        prog='rl.py',
        description='Removes repetitive lines',
    )
    parser.add_argument(
        'filename',
        nargs='?',
        default='',
        help='Input file (default: stdin)'
    )
    parser.add_argument(
        '-o', '--output',
        type=str,
        default='',
        help='Output file (default: stdout)'
    )
    parser.add_argument(
        '-s', '--use-spacing',
        action='store_false',
        help='Uses the leading and trailing whitespace while matching'
    )
    parser.add_argument(
        '-e', '--ignores-empty',
        action='store_false',
        help='Considers an empty line as a duplicate'
    )
    mode_group = parser.add_mutually_exclusive_group()
    mode_group.add_argument(
        '-u', '--unique',
        action='store_true',
        help='Print indexes of unique lines'
    )
    mode_group.add_argument(
        '-d', '--duplicate',
        action='store_true',
        help='Print indexes of duplicate lines'
    )
    mode_group.add_argument(
        '-c', '--comment',
        nargs=1,
        type=str,
        help='Comments the duplicated lines'
    )
    args = parser.parse_args()

    # Setup
    lines = sys.stdin
    out_file = sys.stdout
    mode = 'default'
    use_spacing = args.use_spacing
    ignores_empty = args.ignores_empty
    comment = args.comment[0] if args.comment else None

    if args.filename:
        try:
            lines = open(args.filename, 'r')
        except Exception as e:
            sys.exit(f'Error opening input file "{args.filename}": {e}')

    if args.output:
        try:
            out_file = open(args.output, 'w')
        except Exception as e:
            sys.exit(f'Error opening output file "{args.output}": {e}')

    if args.unique:
        mode = 'unique'
    elif args.duplicate:
        mode = 'duplicate'
    elif args.comment:
        mode = 'comment'

    return lines, out_file, mode, use_spacing, ignores_empty, comment

def clean_files(args):
    lines, out_file, *_ = args

    if lines is not sys.stdin:
        lines.close()
    if out_file is not sys.stdout:
        out_file.close()

Lastly, the last remaining part is the entrypoint itself.

In [38]:
if __name__ == '__main__' and 'ipykernel' not in sys.modules:
    args = parse_args()
    out_file = args[1]

    for out in process_lines(*args):
        print(out, file=out_file)

    clean_files(args)